In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.loader import pg, read as pg_read
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import atomic_id
market_code = "google-play"
update_date = "2020-06-30"
PG_INTERFACE_IOS = {"engine": "PG", "database": "aa"}
PG_INTERFACE_GP = {"engine": "PG", "database": "aa_android"}
PUBLISHER_EVENT_SCHEMA = T.StructType([
    T.StructField("id", T.LongType(), False),
    T.StructField("app_id", T.LongType(), False),
    T.StructField("store_id", T.IntegerType(), False),
    T.StructField("date", T.StringType(), False),
    T.StructField("old_value", T.StringType(), False),
    T.StructField("new_value", T.StringType(), False),
])
urn = Urn(namespace="aa.store.app-event.v1")
sql = """
    SELECT id, app_id, store_id, date, old_value, new_value
    FROM {table}
    WHERE type = 33
"""
if market_code == "apple-store":
    interface = PG_INTERFACE_IOS
    sql = sql.format(table="aa_event")
elif market_code == "google-play":
    interface = PG_INTERFACE_GP
    sql = sql.format(table="event")
event_rows = pg_read(
    urn, interface, sql_param_pair=(sql,),
    mode=pg.QueryMode.COPY,
    dataframe_schema=PUBLISHER_EVENT_SCHEMA
)
df = spark.createDataFrame(event_rows, PUBLISHER_EVENT_SCHEMA)
print df.count()
# df.show(10, False)
publisher_event_df = (
    df
    .withColumn("event_type", F.lit("publisher_id"))
    .withColumn("market_code", F.lit(market_code))
    .withColumn("update_date", F.lit(update_date))
    .withColumn("_identifier", F.lit(atomic_id()))
)
print publisher_event_df.count()
publisher_event_df.show(10, False)
publisher_event_df.cache().createOrReplaceTempView("publisher_event")


In [0]:

from bdce.common.utils import update_application_code
update_application_code(
    spark, role="BDP-PROD-INT-DATAPIPELINE", application_name="ap_aa_dna_mapping_log"
)

In [0]:

import json
from aadatapipelinecore.core.fs.device import DynamoBucket
from aadatapipelinecore.core.fs import Conf
app_config_dynamo = DynamoBucket(Conf(bucket_name="B2C-PROD-DATA-PIPELINE_application_config"))

namespace = "aa.dna.mapping-log.v1"
conf = json.loads(app_config_dynamo.get([{"id": namespace}])[0]["config"])
print json.dumps(conf.get(namespace))

# conf = {
#     namespace: {
#   "schedule_module": "core.sched_datapipeline_workflow.DataPipelineScheduler",
#   "amr": {
#     "routine": {
#       "default": {
#         "is_multi_az": "false",
#         "slave_instance_type": "r5.2xlarge",
#         "safe_reserved_capacity": 30,
#         "cluster_name": "zeppelin_cluster",
#         "master_instance_type": "m4.xlarge",
#         "retired_window": 1800,
#         "bdp_version": "5.27.0",
#         "node_count":4,
#         "market": "spot",
#         "category_name": "DATAPIPELINE_DNA_MAPPINGLOG-ADHOC-GAORIHE"
#       }
#     }
#   },
#   "pkg_name": "ap_aa_dna_mapping_log",
#   "schedule_params": {
#     "bypass_job_monitor": True
#   }
# }
# }

# print json.dumps(conf.get(namespace))
# app_config_dynamo.update({"id": namespace}, [{"config": json.dumps(conf)}])

In [0]:
%python
import requests
import json
import time

url = "https://6tx93muepa.execute-api.us-east-1.amazonaws.com/prod/ingest/bounded"
headers = {"Content-Type": "application/json"}

json_data = {
  "job_type": "routine",
  "namespace": "aa.dna.mapping-log.v1",
  "options":{
    "date":"2020-06-29"
  },
  "source": [{}]
}

print json_data
# response = requests.post(url, data=json.dumps(json_data), headers=headers)
# print response.text

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa -p 5432 << EOF

BEGIN;

SELECT 
    *
    --id, app_id, store_id, date, type, old_value, new_value
FROM aa_event
WHERE 
    type = 33
    AND date >= '2020-06-15'
    AND app_id = 350648885
LIMIT 10
;

rollback;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF

BEGIN;

SELECT 
    *
    --count(1)
    --id, app_id, store_id, date, type, old_value, new_value
FROM event
WHERE type = 33 and date='2020-01-01'
LIMIT 10;

rollback;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207  -U app_tomcat -d dna -p 5432 << EOF

BEGIN;

SELECT
    *
    --count(distinct company_id)
    --product_id, unified_product_id, publisher_id, company_id, parent_company_id 
FROM in_app_dna_info_mapping
WHERE product_id = 350648885
ORDER BY last_updated
LIMIT 100
;

rollback;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207  -U app_tomcat -d dna -p 5432 << EOF

BEGIN;

SELECT *
FROM genre_id_product_mapping
LIMIT 10
;


rollback;

EOF

In [0]:
%%sh
# universal_id: company_id
# company_id: publisher_id

PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207  -U app_tomcat -d dna -p 5432 << EOF

BEGIN;

--\x

SELECT *
FROM dna_universal_company_mapping
WHERE confirmed AND NOT disabled
LIMIT 10
;


--SELECT count(1)
--FROM dna_deleted_universal_company_mapping
--LIMIT 1;


rollback;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 23.22.35.207  -U app_tomcat -d dna -p 5432 << EOF

BEGIN;

SELECT 
    id, universal_id, parent_id, start_date, end_date, created_time, last_updated_time, confirmed, disabled
FROM dna_universal_company_parent
WHERE
    universal_id = 1000200000000143
    --AND confirmed = True
    AND disabled = False
ORDER BY start_date DESC
LIMIT 10
;

rollback;

EOF

In [0]:
%%sh
+----------------+
|      company_id|
+----------------+
|1000200000034103|
|1000200000139906|
|1000200000034104|
+----------------+

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT  company_id, pub_id, company_type, date_range, date 
 FROM plproxy.execute_select_nestloop(\$proxy\$ 
  SELECT company_id, pub_id, company_type, date_range, date
    FROM pp.company_publisher_map_0
    where company_id=1000200000333846 
    LIMIT 100;
     \$proxy\$) tbl (company_id BIGINT, pub_id BIGINT, company_type SMALLINT, date_range DATERANGE, date DATE)  LIMIT 100 ;
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF

select * 
from plproxy.execute_select_nestloop(\$proxy\$ 

    select *
    from pp.company_publisher_map_1
    limit 5

\$proxy\$) tbl 
(company_id BIGINT, pub_id BIGINT, company_type SMALLINT, date_range DATERANGE, date DATE)
limit 100
;


EOF

In [0]:
%%sh
PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141  -U citus_bdp_prod_int_datapipeline -d aa_store_db -p 5432 << EOF

BEGIN;

SELECT *
FROM dna.dna_mapping_log_4app_dim_v1
WHERE
    (start_date BETWEEN '2019-12-01' AND '2020-01-01' OR end_date BETWEEN '2019-12-01' AND '2020-01-01')
    AND
    (start_date != '1970-01-01' AND end_date IS NOT NULL)
    AND company_id IS NOT NULL AND parent_company_id IS NOT NULL
ORDER BY app_id, start_date
LIMIT 10
;


rollback;

EOF

In [0]:
%%sh
PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141  -U citus_bdp_prod_int_datapipeline -d aa_store_db -p 5432 << EOF

BEGIN;

SELECT count(1) FROM (
    SELECT 
        --distinct app_id, publisher_id
        --distinct publisher_id, company_id
        distinct company_id, parent_company_id
    FROM dna.dna_mapping_log_4app_dim_v1
    WHERE
        (start_date BETWEEN '2019-12-01' AND '2020-01-01' OR end_date BETWEEN '2019-12-01' AND '2020-01-01')
        AND
        (start_date != '1970-01-01' AND end_date IS NOT NULL)
) a
;

rollback;

EOF

In [0]:
%%sh
PGPASSWORD='PZQYyjRHCXZ23LVQ' psql -h 10.2.6.141  -U citus_bdp_prod_int_datapipeline -d aa_store_db -p 5432 << EOF

BEGIN;

SELECT *
FROM store.store_est_publisher_fact_v1
LIMIT 10
;

rollback;

EOF

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/   --recursive --human --summarize
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=google-play/  --recursive --human --summarize

aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/_delta_log/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/update_date=${date}/  --recursive --human --summarize | sort

# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-publisheer-mapping.v1/dimension/update_date=${date}/  --recursive --human --summarize | sort
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/update_date=${date}/  --recursive --human --summarize | sort
# aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-parent-company-mapping.v1/dimension/update_date=${date}/  --recursive --human --summarize | sort

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/update_date=  --recursive --human --summarize

In [0]:
%%sh
aws s3 cp s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/_delta_log/${json_file}.json - | head -n 20

In [0]:

mapping_df = (
    spark
    .read
    .format("delta")
    .option("versionAsOf", "6")
    .load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/")
    .where("update_date = '2020-06-22'")
    .select(["app_id", "publisher_id", "company_id", "parent_company_id", "start_date", "end_date"])
    .orderBy(["app_id", "start_date"])
)
print mapping_df.count()
mapping_df.show(10, False)
mapping_df.cache().createOrReplaceTempView("mapping_log")

In [0]:

mapping_df.where("publisher_id IS NULL").show(10, False)
mapping_df.where("app_id = 350648885").show(10, False)

In [0]:

mapping_df.where("app_id = 281889893").show(10, False)

In [0]:

print mapping_df.select("app_id").distinct().count()
print mapping_df.select(["app_id", "start_date"]).distinct().count()
print mapping_df.where("start_date = '1970-01-01' AND end_date IS NULL").count()

In [0]:

date_validate = spark.sql("""
SELECT * FROM (
    SELECT
        app_id, start_date, end_date,
        lag(start_date, -1) over (partition by app_id order by start_date) next_start_date
    FROM mapping_log
) a
WHERE
    next_start_date IS NOT NULL AND next_start_date != DATE_ADD(end_date, 1)
    --app_id = 331677319
""")
print date_validate.count()
date_validate.show(10, False)

In [0]:

mapping_df.where("app_id = 331677319").show(10, False)

In [0]:

app_df = (
    spark
    .read
    .format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-publisher-mapping.v1/dimension/")
    .where("update_date = '2020-06-22'")
    .select(["app_id", "publisher_id", "start_date", "end_date"])
    .orderBy(["app_id", "start_date"])
)
print app_df.count()
app_df.show(10, False)
app_df.cache().createOrReplaceTempView("app_mapping")

In [0]:

print app_df.select("app_id").distinct().count()

In [0]:

publisher_df = (
    spark
    .read
    .format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.publisher-company-mapping.v1/dimension/")
    .where("update_date = '2020-06-22'")
    .select(["publisher_id", "company_id"])
    .orderBy(["publisher_id"])
)
print publisher_df.count()
publisher_df.show(10, False)
publisher_df.cache().createOrReplaceTempView("publisher_company_mapping")

In [0]:

company_df = (
    spark
    .read
    .format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-parent-company-mapping.v1/dimension/")
    .where("update_date = '2020-06-22'")
    .select(["company_id", "parent_company_id", "start_date", "end_date"])
    .orderBy(["company_id", "start_date"])
)
print company_df.count()
company_df.show(10, False)
company_df.cache().createOrReplaceTempView("company_parent_mapping")

In [0]:

app_cum = (
    spark
    .read
    .format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-cum.v1/fact/")
    .where("date = '2020-04-18'")
    .select(["app_id"])
)
print app_cum.count()
app_cum.show(10, False)
app_cum.cache().createOrReplaceTempView("app_cum")

In [0]:

# apps that in app-publisher mapping, but not in final mapping
app_diff = spark.sql("""
SELECT * 
FROM app_mapping
WHERE app_id IN (
    SELECT app_pub_mapping_app_id AS app_id
    FROM (
        SELECT final.app_id AS final_mapping_app_id, mid.app_id AS app_pub_mapping_app_id
        FROM mapping_log AS final
        FULL JOIN app_mapping AS mid
        ON final.app_id = mid.app_id
    ) a
    WHERE final_mapping_app_id IS NULL
)
""")
print app_diff.count()
app_diff.show(10, False)

In [0]:

app_diff.cache().createOrReplaceTempView("app_diff")
df = spark.sql("""
SELECT final.*
FROM app_diff AS diff
JOIN final_mapping AS final
ON diff.app_id = final.app_id
""")
print df.count()
df.show(10, False)

In [0]:

df.where("end_date IS NOT NULL").show(10, False)

In [0]:

from aadatapipelinecore.core.urn import Urn
from applications.common.test import read_unified_dataframe

SYS_VIEWS = {
    "uds_view_dna_app_publisher_map": {
        "urn": Urn(namespace="aa.dna.app-publisher-mapping.v1", data_type="dimension"),
        "test_data": [
            {
                "date": "2020-04-18",
                "app_id": 1001
            }
        ]
    }
}


def init_single_view(spark, view):
    from aadatapipelinecore.core.log import logger
    urn = SYS_VIEWS[view]["urn"]
    sql_where = SYS_VIEWS[view].get("partition_sql_where", " 1=1 ")
    is_delta_lake = SYS_VIEWS[view].get("is_delta_lake", True)
    is_snapshot = SYS_VIEWS[view].get("is_snapshot", False)
    logger.info("view {} sql where is {}".format(view, sql_where))
    read_unified_dataframe(urn, spark, sql_where, is_delta_lake, is_snapshot).createOrReplaceTempView(view)

view = "uds_view_dna_app_publisher_map"
init_single_view(spark, view)

In [0]:

spark.sql("select count(1) from uds_view_dna_app_publisher_map").show()
spark.sql("select count(distinct app_id) from uds_view_dna_app_publisher_map").show()
spark.sql("select * from uds_view_dna_app_publisher_map limit 10").show(10, False)

In [0]:

spark.sql("""
    SELECT app_id, publisher_id, start_date, end_date
    FROM uds_view_dna_app_publisher_map
    WHERE update_date = '2020-06-22'
""").cache().createOrReplaceTempView("app_publisher_mapping")

In [0]:

spark.sql("""
    SELECT a.app_id, a.publisher_id, b.company_id, a.start_date, a.end_date
    FROM app_publisher_mapping AS a
    LEFT JOIN publisher_company_mapping AS b
    ON a.publisher_id = b.publisher_id
""").cache().createOrReplaceTempView("app_company_mapping")

In [0]:

spark.sql("""
    SELECT DISTINCT
        b.app_id, a.company_id, a.parent_company_id, a.start_date, a.end_date
    FROM app_company_mapping AS b
    LEFT JOIN company_parent_mapping AS a
    ON
        a.company_id = b.company_id
        AND (
            a.start_date BETWEEN b.start_date AND b.end_date
            OR a.end_date BETWEEN b.start_date AND b.end_date
            OR (b.start_date IS NULL AND a.start_date <= b.end_date)
            OR (b.end_date IS NULL AND a.end_date >= b.start_date)
            OR (b.start_date IS NULL AND b.end_date IS NULL)
        )
""").cache().createOrReplaceTempView("app_parent_company_mapping")

In [0]:

spark.sql("""
    SELECT DISTINCT app_id, date FROM (
        SELECT app_id, CASE WHEN start_date IS NULL THEN '1970-01-01' ELSE start_date END AS date FROM app_company_mapping
        UNION
        SELECT app_id, DATE_ADD(end_date, 1) AS date FROM app_company_mapping
        UNION
        SELECT app_id, CASE WHEN start_date IS NULL THEN '1970-01-01' ELSE start_date END AS date FROM app_parent_company_mapping
        UNION
        SELECT app_id, DATE_ADD(end_date, 1) AS date FROM app_parent_company_mapping
    ) a
    WHERE date IS NOT NULL
    ORDER BY app_id, date
""").cache().createOrReplaceTempView("all_dates")

In [0]:

spark.sql("""
    SELECT app_id, date AS start_date, date_add(lead(date, 1) over (partition by app_id order by date), -1) AS end_date
    FROM all_dates
    --UNION
    --SELECT app_id, null AS start_date, date_add(first_value(date) over (partition by app_id order by date), -1) AS end_date
    --FROM all_dates
""").cache().createOrReplaceTempView("all_date_range")

In [0]:

spark.sql("""
    SELECT
        company.app_id, company.publisher_id, company.company_id, parent.parent_company_id,
        --CASE WHEN company.start_date IS NULL THEN '1970-01-01' ELSE company.start_date END AS start_date,
        company.start_date,
        company.end_date,
        '2020-06-23' AS update_date
    FROM
        (
            SELECT
                range.app_id, map.publisher_id, map.company_id, range.start_date, range.end_date
            FROM all_date_range AS range
            LEFT JOIN app_company_mapping AS map
            ON
                map.app_id = range.app_id
                AND (range.start_date >= map.start_date OR map.start_date IS NULL)
                AND (range.end_date <= map.end_date OR map.end_date IS NULL)
        ) AS company
        LEFT JOIN company_parent_mapping AS parent
        ON
            company.company_id = parent.company_id
            AND (company.start_date >= parent.start_date OR parent.start_date IS NULL)
            AND (company.end_date <= parent.end_date OR parent.end_date IS NULL)
""").cache().createOrReplaceTempView("final_mapping")

In [0]:

spark.sql("select * from final_mapping where app_id = 285735373").show(10, False)
spark.sql("select * from final_mapping where app_id = 331677319").show(10, False)
# spark.sql("select count(1) from all_dates").show(10, False)

In [0]:

spark.sql("select count(1) from final_mapping where start_date IS NULL").show(10, False)

In [0]:

spark.sql("select count(1) from final_mapping").show(10, False)
spark.sql("select count(distinct app_id) from final_mapping").show(10, False)
spark.sql("select count(1) from final_mapping WHERE start_date = '1970-01-01' AND end_date IS NULL").show(10, False)

In [0]:

spark.sql("select * from final_mapping WHERE publisher_id IS NULL").show(10, False)
spark.sql("select * from app_publisher_mapping WHERE publisher_id IS NULL").show(10, False)

In [0]:

check = spark.sql("""
SELECT * FROM (
    SELECT
        app_id, start_date, end_date,
        lag(start_date, -1) over (partition by app_id order by start_date) next_start_date
    FROM mapping_log
) a
WHERE
    next_start_date IS NOT NULL AND next_start_date != DATE_ADD(end_date, 1)
    --app_id = 331677319
""")
print check.count()
check.show(10, False)

In [0]:

mapping_df.where("app_id = 20600007740853").show(10, False)
mapping_df.where("app_id = 20600006822910").show(10, False)
mapping_df.where("app_id = 1466586824").show(10, False)

In [0]:

spark.sql("""
SELECT 
    DISTINCT publisher_id, company_id
FROM historical_mapping
WHERE 
    publisher_id IN (542280115, 20200003115819)
""").show()

In [0]:

mapping_df.where("parent_company_id = 1000200000031106").where("start_date < '2018-06-15'").show(100, False)

In [0]:

print mapping_df.where("app_id IS NULL").count()
print mapping_df.select("app_id").distinct().count()
# print publisher_df.select("app_id").distinct().count()

In [0]:

publisher_df = (
    spark.read.format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-publisher-mapping.v1/dimension/")
    .where("update_date = '2020-06-15'")
    .select(["app_id", "publisher_id", "start_date", "end_date"])
    .orderBy(["app_id", "start_date"])
)
print publisher_df.count()
publisher_df.show(10, False)
publisher_df.cache().createOrReplaceTempView("publisher_df")

In [0]:

print publisher_df.select("app_id").distinct().count()

In [0]:

publisher_df.where("end_date = '2014-04-08' AND start_date IS NULL").show(10, False)

In [0]:

company_df = (
    spark.read.format("delta")
    .load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.company-parent-company-mapping.v1/dimension/")
    .where("update_date = '2020-06-15'")
    .select(["company_id", "parent_company_id", "start_date", "end_date"])
    .orderBy(["company_id", "start_date"])
)
print company_df.count()
company_df.show(10, False)
company_df.cache().createOrReplaceTempView("parent_mapping")

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.loader import pg, read as pg_read
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import atomic_id

update_date = "2020-06-16"

interface = {"engine": "PG", "database": "dna"}

DNA_PUBLISHER_COMPANY_SCHEMA = T.StructType([
    T.StructField("universal_id", T.LongType(), False),
    T.StructField("company_id", T.LongType(), False),
    T.StructField("last_updated", T.StringType(), False),
])

urn = Urn(namespace="aa.store.dna-company.v1")
sql = """
    SELECT DISTINCT universal_id, company_id, last_updated_time as last_updated
    FROM {table}
    WHERE confirmed AND NOT disabled
""".format(table = "dna_universal_company_mapping")

dna_rows = pg_read(
    urn, interface, sql_param_pair=(sql,),
    mode=pg.QueryMode.COPY,
    dataframe_schema=DNA_PUBLISHER_COMPANY_SCHEMA
)
df = spark.createDataFrame(dna_rows, DNA_PUBLISHER_COMPANY_SCHEMA)

print df.count()

dna_publisher_df = (
    df
    .withColumnRenamed("company_id", "publisher_id")
    .withColumnRenamed("universal_id", "company_id")
    .withColumn("update_date", F.lit(update_date))
    .withColumn("_identifier", F.lit(atomic_id()))
)
print dna_publisher_df.count()
dna_publisher_df.show(10, False)
dna_publisher_df.cache().createOrReplaceTempView("dna_df")

In [0]:

left = spark.sql("""
    SELECT a.app_id, a.publisher_id, b.company_id, a.start_date, a.end_date
    FROM publisher_mapping AS a
    LEFT JOIN dna_df AS b
    ON a.publisher_id = b.publisher_id
""")
print left.count()
left.show(10, False)
left.cache().createOrReplaceTempView("app_company_mapping")

In [0]:

app_parent_company = spark.sql("""
    SELECT b.app_id, a.company_id, a.parent_company_id, a.start_date, a.end_date
    FROM parent_mapping AS a
    JOIN app_company_mapping AS b
    ON
        a.company_id = b.company_id
        AND (
            a.start_date BETWEEN b.start_date AND b.end_date
            OR a.end_date BETWEEN b.start_date AND b.end_date
            OR (b.start_date IS NULL AND a.start_date <= b.end_date)
            OR (b.end_date IS NULL AND a.end_date >= b.start_date)
        )
""")
app_parent_company.cache().createOrReplaceTempView("app_parent_company")
all_dates = spark.sql("""
SELECT DISTINCT app_id, date FROM (
    SELECT app_id, start_date AS date FROM app_company_mapping WHERE company_id IS NOT NULL
    UNION
    SELECT app_id, DATE_ADD(end_date, 1) AS date FROM app_company_mapping WHERE company_id IS NOT NULL
    UNION 
    SELECT app_id, start_date AS date FROM app_parent_company
    UNION
    SELECT app_id, DATE_ADD(end_date, 1) AS date FROM app_parent_company
) a
WHERE date IS NOT NULL
ORDER BY app_id, date
""")
print all_dates.count()
all_dates.show(10, False)
all_dates.cache().createOrReplaceTempView("all_dates")

In [0]:

all_date_range = spark.sql("""
SELECT app_id, date AS start_date, date_add(lead(date, 1) over (partition by app_id order by date), -1) AS end_date
FROM all_dates
UNION
SELECT app_id, null AS start_date, date_add(first_value(date) over (partition by app_id order by date), -1) AS end_date
FROM all_dates
""")
print all_date_range.count()
all_date_range.show(10, False)
all_date_range.cache().createOrReplaceTempView("all_date_range")

In [0]:

app_parent_company.where("app_id = 281889893").orderBy("start_date").show(10, False)

# app_company_mapping
left.where("app_id = 281889893").orderBy("start_date").show(10, False)

# parent_mapping
company_df.where("company_id IN (1000200000003452, 1000200000003455)").show(10, False)

In [0]:

# 281889893, 319581197
# new_app_company_mapping.where("app_id = 319581197").orderBy("start_date").show(10, False)
result.where("app_id = 281889893").orderBy("start_date").show(10, False)

In [0]:

all_dates.where("app_id = 319581197").orderBy("date").show(10, False)
all_date_range.where("app_id = 319581197").orderBy("start_date").show(10, False)

In [0]:

# all_date_range - range
+---------+----------+----------+
|app_id   |start_date|end_date  |
+---------+----------+----------+
|319581197|null      |2014-07-06|
|319581197|2014-07-07|2015-06-18|
|319581197|2015-06-19|2018-01-31|
|319581197|2018-02-01|2020-01-22|
|319581197|2020-01-23|null      |
+---------+----------+----------+

# app_company_mapping - map
+---------+------------+----------------+----------+----------+
|app_id   |publisher_id|company_id      |start_date|end_date  |
+---------+------------+----------------+----------+----------+
|319581197|304712583   |1000200000034131|null      |2015-06-18|
|319581197|978181908   |1000200000034131|2015-06-19|2020-01-22|
|319581197|304712583   |1000200000034131|2020-01-23|null      |
+---------+------------+----------------+----------+----------+

In [0]:

new_app_company_mapping = spark.sql("""
        SELECT
            map.app_id, map.publisher_id, map.company_id, range.start_date, range.end_date
        FROM all_date_range AS range
        LEFT JOIN app_company_mapping AS map
        ON
            map.app_id = range.app_id 
            AND 
            ( 
                (range.start_date >= map.start_date OR map.start_date IS NULL)
                AND (range.end_date <= map.end_date OR map.end_date IS NULL)
            )
""")

In [0]:

# all_date_range
# app_company_mapping
# parent_mapping

result = spark.sql("""
SELECT 
    company.app_id, company.publisher_id, company.company_id, parent.parent_company_id, company.start_date, company.end_date
FROM
    (
        SELECT
            map.app_id, map.publisher_id, map.company_id, range.start_date, range.end_date
        FROM all_date_range AS range
        LEFT JOIN app_company_mapping AS map
        ON
            map.app_id = range.app_id 
            AND 
            (range.start_date >= map.start_date OR map.start_date IS NULL)
            AND (range.end_date <= map.end_date OR map.end_date IS NULL)
    ) AS company
    LEFT JOIN parent_mapping AS parent
    ON
        company.company_id = parent.company_id 
        AND (company.start_date >= parent.start_date OR parent.start_date IS NULL)
        AND (company.end_date <= parent.end_date OR parent.end_date IS NULL)
""")
print result.count()
result.show(10, False)

In [0]:

validate = spark.sql("""
SELECT * FROM (
    SELECT
        app_id, start_date, end_date,
        lag(start_date, -1) over (partition by app_id order by start_date) next_start_date
    FROM app_company_mapping
) a
WHERE
    next_start_date IS NOT NULL AND next_start_date != DATE_ADD(end_date, 1)
    --app_id = 331677319
""")
print validate.count()
validate.show(10, False)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/ --recursive

In [0]:

event = (
    spark
    .read
    .option("basePath", "s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/")
    .parquet("s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/update_date=2020-06-11/")
    .select(["market_code", "id", "app_id", "store_id", "date", "old_value", "new_value"])
)
print event.count()
event.show(10, False)
event.cache().createOrReplaceTempView("event")

In [0]:

spark.sql("""
SELECT * FROM event
WHERE
    app_id = 20600009608756
    AND date = '2019-06-17 07:00:00+00'
ORDER BY id DESC
""").show(10, False)

In [0]:

t = spark.sql("""
SELECT * FROM (
    SELECT market_code, app_id, store_id, date, count(1) AS count
    FROM event
    GROUP BY market_code, app_id, store_id, date
) a
WHERE count > 1
ORDER BY count DESC
""")
print t.count()
t.show(100, False)

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.loader import pg, read as pg_read
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import atomic_id

market_code = "google-play"
update_date = "2020-06-11"


PG_INTERFACE_IOS = {"engine": "PG", "database": "aa"}
PG_INTERFACE_GP = {"engine": "PG", "database": "aa_android"}

PUBLISHER_EVENT_SCHEMA = T.StructType([
    T.StructField("id", T.LongType(), False),
    T.StructField("app_id", T.LongType(), False),
    T.StructField("store_id", T.IntegerType(), False),
    T.StructField("date", T.StringType(), False),
    T.StructField("old_value", T.StringType(), False),
    T.StructField("new_value", T.StringType(), False),
])

urn = Urn(namespace="aa.store.app-event.v1")
sql = """
    SELECT id, app_id, store_id, date, old_value, new_value
    FROM {table}
    WHERE type = 33
"""

if market_code == "apple-store":
    interface = PG_INTERFACE_IOS
    sql = sql.format(table="aa_event")
elif market_code == "google-play":
    interface = PG_INTERFACE_GP
    sql = sql.format(table="event")

event_rows = pg_read(
    urn, interface, sql_param_pair=(sql,),
    mode=pg.QueryMode.COPY,
    dataframe_schema=PUBLISHER_EVENT_SCHEMA
)
df = spark.createDataFrame(event_rows, PUBLISHER_EVENT_SCHEMA)

print df.count()
# df.show(10, False)


publisher_event_df = (
    df
    .withColumn("event_type", F.lit("publisher_id"))
    .withColumn("market_code", F.lit(market_code))
    .withColumn("update_date", F.lit(update_date))
    .withColumn("_identifier", F.lit(atomic_id()))
)
print publisher_event_df.count()
publisher_event_df.show(10, False)
publisher_event_df.cache().createOrReplaceTempView("publisher_event")

In [0]:

# publisher_event_df.coalesce(2).write.parquet(
    "s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/",
    mode="append",
    compression="gzip",
    partitionBy=["update_date", "market_code", "event_type"]

)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/update_date=

In [0]:

from pyspark.sql.functions import lit

publisher_event = (
    spark.read
    .option("basePath", "s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/")
    .parquet("s3://b2c-prod-data-pipeline-unified-store-free/_obsolete/store.app-event.v1/fact/update_date={date}/".format(date="2020-06-11"))
    .drop("_identifier")
)
print publisher_event.count()
publisher_event.show(10, False)
publisher_event.cache().createOrReplaceTempView("publisher_event")

In [0]:

df = spark.sql("""
    SELECT
        market_code, app_id, date, old_value, new_value,
        ROW_NUMBER() OVER (PARTITION BY app_id ORDER BY date) AS rn
    FROM (
        SELECT DISTINCT
            'apple-store' AS market_code, app_id, date_format(date, 'yyyy-MM-dd') AS date,
            FIRST_VALUE(old_value) OVER (PARTITION BY app_id, date_format(date, 'yyyy-MM-dd') ORDER BY id ASC) AS old_value,
            FIRST_VALUE(new_value) OVER (PARTITION BY app_id, date_format(date, 'yyyy-MM-dd') ORDER BY id DESC) AS new_value
        FROM publisher_event
    ) AS a
    WHERE
        old_value != new_value
""")
print df.count()
df.show(10, False)
df.cache().createOrReplaceTempView("agg_publisher_event")

In [0]:

spark.sql("""
    SELECT a.market_code, a.app_id, a.new_value AS publisher_id, a.date AS start_date, DATE_ADD(b.date, -1) AS end_date
    FROM agg_publisher_event AS a
    LEFT JOIN agg_publisher_event AS b
    ON
        a.app_id = b.app_id
        AND a.rn + 1 = b.rn
    UNION ALL
    SELECT market_code, app_id, old_value AS publisher_id, NULL AS start_date, DATE_ADD(date, -1) AS end_date
    FROM agg_publisher_event
    WHERE rn = 1
""").cache().createOrReplaceTempView("publisher_change_log")

In [0]:

spark.sql("""
    SELECT
        market_code,
        app_id,
        new_value AS publisher_id,
        date AS start_date,
        DATE_ADD(LAG(date, 1) OVER (PARTITION BY app_id ORDER BY date DESC), -1) AS end_date
    FROM agg_publisher_event
    UNION ALL
    SELECT DISTINCT
        market_code,
        app_id,
        FIRST_VALUE(old_value) OVER (PARTITION BY app_id ORDER BY date) AS publisher_id,
        NULL AS start_date,
        DATE_ADD(FIRST_VALUE(date) OVER (PARTITION BY app_id ORDER BY date), -1) AS end_date
    FROM agg_publisher_event
""").cache().createOrReplaceTempView("publisher_change_log")

In [0]:

spark.sql("select * from publisher_change_log where app_id = 319581197").orderBy("start_date").show(10, False)

In [0]:

df.where("app_id = 319581197").orderBy("date").show(10, False)

In [0]:

a
+-----------+---------+----------+---------+---------+---+
|market_code|app_id   |date      |old_value|new_value|rn |
+-----------+---------+----------+---------+---------+---+
|apple-store|319581197|2015-06-19|304712583|978181908|1  |
|apple-store|319581197|2020-01-23|978181908|304712583|2  |
+-----------+---------+----------+---------+---------+---+

+---------+-----------+------------+------------+----------+----------+
|app_id   |market_code|event_type  |publisher_id|start_date|end_date  |
+---------+-----------+------------+------------+----------+----------+
|319581197|apple-store|publisher_id|304712583   |null      |2015-06-18|
|319581197|apple-store|publisher_id|978181908   |2015-06-19|2020-01-22|
|319581197|apple-store|publisher_id|304712583   |2020-01-23|null      |
+---------+-----------+------------+------------+----------+----------+

In [0]:

t1 = spark.sql("""
SELECT app_id, date, old_value, new_value, ROW_NUMBER() OVER (PARTITION BY app_id ORDER BY date) AS rn
FROM agg_publisher_event
""")
t1.cache().createOrReplaceTempView("t")

test = spark.sql("""
SELECT a.app_id, a.new_value AS publisher_id, a.date AS start_date, b.date AS end_date
FROM t AS a
LEFT JOIN t AS b
ON
    a.app_id = b.app_id
    AND a.rn + 1 = b.rn
UNION ALL
SELECT app_id, old_value AS publisher_id, NULL AS start_date, date AS end_date
FROM t
WHERE rn = 1
""").orderBy(["app_id", "start_date"])
print test.count()
test.show(10, False)
test.cache().createOrReplaceTempView("publisher_change_log")

In [0]:

publisher_mapping = spark.sql("""
SELECT
    app_id, publisher_id, start_date, end_date
FROM publisher_change_log
UNION ALL
SELECT 
    app_id, publisher_id, NULL AS start_date, NULL AS end_date
FROM publisher_df
WHERE
    app_id NOT IN (
        SELECT DISTINCT app_id
        FROM publisher_change_log
    )
""")
print publisher_mapping.count()
publisher_mapping.show(10, False)

In [0]:

(
    # publisher_mapping
    .coalesce(10)
    .write
    .format("delta")
    .mode("append")
    .save("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app-publisher-mapping.v1/dimension/")
)

In [0]:

diff = publisher_mapping.where("is_publisher_same = False AND start_date < '2020-06-01'")
print diff.count()

diff.select(["market_code", "status"]).groupby(["market_code", "status"]).count().orderBy(["market_code", "status"]).show(10, False)
diff.show(10, False)

In [0]:

publisher_diff = spark.sql("""
SELECT DISTINCT
    detail.market_code, detail.app_id, detail.status, detail.publisher_id, change.event_type, change.date, change.new_value,
    CASE WHEN detail.publisher_id = change.new_value THEN True ELSE False END AS is_publisher_same
FROM (
    SELECT market_code, app_id, event_type, update_time AS date, new_value
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY app_id, market_code, event_type ORDER BY update_time DESC, id DESC) AS rn
        FROM (
            SELECT id, app_id, market_code, event_type, date_format(date, 'yyyy-MM-dd') AS date, date AS update_time, old_value, new_value
            FROM publisher_event
        ) a
    ) b
    WHERE rn = 1
) AS change
JOIN
(
    SELECT DISTINCT market_code, app_id, status, publisher_id
    FROM publisher_df
) AS detail
ON
    change.app_id = detail.app_id
""").where("is_publisher_same = False AND date <= '2020-06-01'").orderBy(["market_code", "date"])
print publisher_diff.count()
publisher_diff.show(10, False)

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa_android -p 5433 << EOF

BEGIN;

SELECT id, name, company_id, status
FROM app
WHERE 
    id = 20600009608756
LIMIT 10
;

SELECT *
FROM event
WHERE 
    app_id = 20600009608756 AND type = 33
    AND date < '2019-06-20'
ORDER BY id DESC
LIMIT 100
;

rollback;

EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2  -U app_tomcat -d aa -p 5432 << EOF

BEGIN;

SELECT id, name, company_id, status
FROM aa_app
WHERE 
    id = 328171271
LIMIT 10
;

SELECT *
FROM aa_event
WHERE 
    app_id = 328171271 AND type = 33
ORDER BY date DESC
LIMIT 10
;

\d aa_event

END;

EOF

In [0]:
%%sh
